<a href="https://colab.research.google.com/github/preetamjumech/LLM/blob/main/Rag_Ollama3_FAISS_22_10_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install langchain_community==0.2.4
#!pip install langchain==0.2.3
#!pip install faiss_cpu==1.8.0
#!pip install unstructured==0.14.5
#!pip install unstructured[pdf]==0.14.5
#!pip install transformers==4.41.2
#!pip install sentence_transformers==3.0.1
#!pip install streamlit

In [ ]:
import streamlit as st
from langchain_community.llms import Ollama
from langchain.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains import RetrievalQA


llm = Ollama(model="llama3:instruct",
             temperature = 0)
# Set page configuration


st.set_page_config(page_title="Document Q&A App", layout="wide")
col1, col2 = st.columns([1, 4])  # Adjust the ratios as needed

with col1:
    st.image("logo.png")

with col2:
    st.markdown("<h1 style='text-align: center;'>🧠 RAG : Q&A with your own Documents</h1>", unsafe_allow_html=True)


# Sidebar for instructions and steps with increased font size
st.sidebar.markdown("<h2 style='font-size: 20px;'>General Instructions</h2>", unsafe_allow_html=True)
st.sidebar.markdown("<h4 style='font-size: 16px;'>Steps to Use the App:</h4>", unsafe_allow_html=True)
st.sidebar.markdown("1. Upload your PDF document.")
st.sidebar.markdown("2. Create embeddings from the document.")
st.sidebar.markdown("3. Ask questions based on the document content.")

# Adding some spacing before the main content
st.markdown("<br>", unsafe_allow_html=True)

# Section 1: Document Upload
st.header("📄 Upload Document ")
uploaded_file = st.file_uploader("Upload a PDF document", type="pdf")
if uploaded_file is not None:
    # Save the uploaded file temporarily
    with open("uploaded_document.pdf", "wb") as f:
        f.write(uploaded_file.getbuffer())
    st.success("✅ Document uploaded!")

    # Clear embeddings message
    if 'embeddings_status' in st.session_state:
        del st.session_state['embeddings_status']

# Adding extra spacing
st.markdown("<br>", unsafe_allow_html=True)

# Section 2: Create Embeddings
st.header("🔗 Create Embeddings")
if st.button("Create Embeddings") and uploaded_file is not None:
    loader = UnstructuredFileLoader("uploaded_document.pdf")
    documents = loader.load()

    text_splitter = CharacterTextSplitter(separator="/n", chunk_size=1000, chunk_overlap=250)
    texts = text_splitter.split_documents(documents)

    embeddings = OllamaEmbeddings(model="nomic-embed-text")
    knowledge_base = FAISS.from_documents(texts, embeddings)

    # Store knowledge_base in session state
    st.session_state.knowledge_base = knowledge_base

    # Set embeddings status
    st.session_state.embeddings_status = "✨ Embeddings created!"

# Display embeddings status
# Display embeddings status
if 'embeddings_status' in st.session_state:
    st.markdown(
        "<h3 style='color: green;'>✅ Embeddings created!</h3>",
        unsafe_allow_html=True
    )

# Adding some spacing between sections
st.markdown("<br>", unsafe_allow_html=True)

# Section 3: Question Answering
st.header("❓ Question Answering")
question = st.text_input("Ask a question about the document")
if st.button("Get Answer") and question and 'knowledge_base' in st.session_state:
    qa_chain = RetrievalQA.from_chain_type(llm, retriever=st.session_state.knowledge_base.as_retriever())
    response = qa_chain.invoke({"query": question})
    st.write("📝 Answer:", response["result"])
